## 获取IP代理地址

+ 获取网站<https://www.xicidaili.com/>
+ 查看提取目标信息的xpath

![f12](https://user-images.githubusercontent.com/62495140/80340242-dd711580-8892-11ea-8171-5014f6884f13.jpg)

copy的ip的xpath为：//*[@id="ip_list"]/tbody/tr[3]/td[2] 

注意，该xpath不可用，因为这是浏览器自动生成的，在Network的Response中查看html代码

![network](https://user-images.githubusercontent.com/62495140/80340253-e235c980-8892-11ea-84b5-6b8137e80cf1.jpg)

真正的ip的xpath为：//*[@id="ip_list"]/tr/td[2]/text()


In [15]:
from lxml import etree
import requests
import time
import pickle

In [13]:
def open_proxy_url(url):
    # f12查看网站的头信息
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36',}
    try:
        r = requests.get(url,headers=headers,timeout=20)
        r.raise_for_status()
        r.encoding = r.apparent_encoding
        return r.text
    except:
        print('无法访问网页：' + url)
        
        
def get_proxy_ip(html):
    tree = etree.HTML(html)
    ip = tree.xpath('//*[@id="ip_list"]/tr/td[2]/text()')#ip
    ip_port = tree.xpath('//*[@id="ip_list"]/tr/td[3]/text()')#端口号
    ip_type = tree.xpath('//*[@id="ip_list"]/tr/td[6]/text()')#网络类型
    
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
    headers = {'User_Agent':user_agent}
    
    proxy_ip_lis = []
    for i,j,k in zip(ip,ip_port,ip_type):
        try:
            proxies = {
                    k.lower():i+':'+j  # 注意到爬下来的是HTTP或者是HTTPS，我们要将它们变成http或https
                    }
            print(proxies)  # 把IP输出来看看
            
            # 由于爬取的ip是免费的，质量不稳定，要进行测试是否可用：用这个IP发一个请求给百度，测试一下这个IP能不能用
            requests.get('https://www.baidu.com/',headers=headers,timeout=1,proxies=proxies)          
            proxy_ip_lis.append(proxies)  # 如果没有超时，说明能用，存进列表里
            print('http://'+i+':'+j+" is fine")
        except:
            print('http://'+i+':'+j+" is timeout")  # 超时抛出错误，输出哪个IP超时
    return proxy_ip_lis  # 返回可用IP列表

In [16]:
if __name__ == '__main__':
    url = 'https://www.xicidaili.com/'
    html = open_proxy_url(url)
    print(type(html))
    lis = get_proxy_ip(html)
    pickle.dump(lis,open('ip.pkl','wb'))#将可用的IP进行存储，以为了方便其他程序调用，供其他程序读取、使用
    print('----可用IP---')
    for item in lis:
        print(item)
    print('---  END  ---')

<class 'str'>
{'https': '118.24.118.207:808'}
http://118.24.118.207:808 is timeout
{'http': '101.132.123.99:8080'}
http://101.132.123.99:8080 is fine
{'https': '114.223.103.47:8118'}
http://114.223.103.47:8118 is timeout
{'http': '112.192.34.47:1133'}
http://112.192.34.47:1133 is fine
{'http': '115.29.170.58:8118'}
http://115.29.170.58:8118 is fine
{'https': '58.254.220.116:52470'}
http://58.254.220.116:52470 is timeout
{'http': '223.100.166.3:36945'}
http://223.100.166.3:36945 is fine
{'https': '171.35.166.86:9999'}
http://171.35.166.86:9999 is timeout
{'https': '110.243.8.26:9999'}
http://110.243.8.26:9999 is timeout
{'https': '119.84.112.139:80'}
http://119.84.112.139:80 is timeout
{'https': '119.84.112.137:80'}
http://119.84.112.137:80 is timeout
{'http': '60.217.129.63:8060'}
http://60.217.129.63:8060 is fine
{'https': '139.196.193.85:8080'}
http://139.196.193.85:8080 is timeout
{'http': '101.132.190.101:80'}
http://101.132.190.101:80 is fine
{'http': '182.138.160.189:8118'}
http: